In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"parnikagup","key":"b510062a30729c42e67073b9b8e6daaf"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio
!unzip ravdess-emotional-speech-audio.zip -d dataset
!ls dataset

Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
ravdess-emotional-speech-audio.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ravdess-emotional-speech-audio.zip
replace dataset/Actor_01/03-01-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace dataset/Actor_01/03-01-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: dataset/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: dataset/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: dataset/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: dataset/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: dataset/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: dataset/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: dataset/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: dataset/Actor_01/03-01-02-02-01-01-01.wav

In [6]:
import librosa

sample = "dataset/Actor_01/03-01-01-01-01-01-01.wav"
y, sr = librosa.load(sample)

print("Audio loaded:", y.shape, "Sample rate:", sr)

Audio loaded: (72838,) Sample rate: 22050


In [12]:
import os
import numpy as np
import librosa
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
def extract_mfcc(file_path, max_len=130):
    y, sr = librosa.load(file_path, duration=3)

    # MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

    # Delta & Delta-Delta
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)

    # Stack → (120, T)
    mfcc = np.vstack([mfcc, delta, delta2])

    # Normalize
    mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)

    # Pad / truncate time axis
    if mfcc.shape[1] < max_len:
        mfcc = np.pad(mfcc, ((0,0),(0,max_len - mfcc.shape[1])))
    else:
        mfcc = mfcc[:, :max_len]

    return mfcc

In [14]:
emotion_map = {
    "01": 0,  # neutral
    "02": 1,  # calm
    "03": 2,  # happy
    "04": 3,  # sad
    "05": 4,  # angry
    "06": 5,  # fearful
    "07": 6,  # disgust
    "08": 7   # surprised
}

In [15]:
X, y = [], []

for actor in os.listdir("dataset"):
    actor_path = os.path.join("dataset", actor)

    if not os.path.isdir(actor_path):
        continue

    for file in os.listdir(actor_path):
        if not file.endswith(".wav"):
            continue

        parts = file.split("-")
        if len(parts) < 3:
            continue

        emotion = parts[2]

        X.append(extract_mfcc(os.path.join(actor_path, file)))
        y.append(emotion_map[emotion])

X = np.array(X)
y = np.array(y)

print("Data shape:", X.shape)   # (samples, 120, 130)

Data shape: (1440, 120, 130)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [17]:
class EmotionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(
    EmotionDataset(X_train, y_train),
    batch_size=32,
    shuffle=True
)

test_loader = DataLoader(
    EmotionDataset(X_test, y_test),
    batch_size=32,
    shuffle=False
)

In [18]:
class EmotionCNNLSTM(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)

        self.lstm = nn.LSTM(
            input_size=64 * 28,
            hidden_size=128,
            batch_first=True
        )

        self.fc = nn.Linear(128, 8)

    def forward(self, x):
        x = x.unsqueeze(1)          # (B,1,120,130)
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))

        # (B,64,28,31) → (B,31,64*28)
        x = x.permute(0, 3, 1, 2)
        x = x.contiguous().view(x.size(0), x.size(1), -1)

        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1])


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionCNNLSTM().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/20 | Loss: {running_loss:.4f}")

Epoch 1/30 | Loss: 70.8088
Epoch 2/30 | Loss: 62.0853
Epoch 3/30 | Loss: 56.9828
Epoch 4/30 | Loss: 52.6181
Epoch 5/30 | Loss: 49.9935
Epoch 6/30 | Loss: 47.4887
Epoch 7/30 | Loss: 43.1647
Epoch 8/30 | Loss: 41.2797
Epoch 9/30 | Loss: 37.1647
Epoch 10/30 | Loss: 34.7478
Epoch 11/30 | Loss: 32.5083
Epoch 12/30 | Loss: 27.8682
Epoch 13/30 | Loss: 27.4535
Epoch 14/30 | Loss: 21.7310
Epoch 15/30 | Loss: 18.0347
Epoch 16/30 | Loss: 15.6140
Epoch 17/30 | Loss: 14.1980
Epoch 18/30 | Loss: 13.2939
Epoch 19/30 | Loss: 10.3012
Epoch 20/30 | Loss: 7.5758


In [20]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        outputs = model(xb)
        preds = torch.argmax(outputs, dim=1)

        y_true.extend(yb.numpy())
        y_pred.extend(preds.cpu().numpy())

acc = accuracy_score(y_true, y_pred)
print("Test Accuracy:", acc)

Test Accuracy: 0.7152777777777778
